# Crescent Moon Visibility Calculator

This notebook calculates and visualizes crescent moon visibility for any year using the Yallop method.

## Features
- Calculate new moon dates for any year
- Generate high-resolution (4K) visibility maps
- Process multiple days after each new moon
- Display results inline in the notebook

## Setup and Installation

In [57]:
# Install required packages
!pip install ephem pillow -q

# Check if ImageMagick is installed and install if needed
import subprocess
try:
    subprocess.run(['magick', '--version'], check=True, capture_output=True)
    print("✓ ImageMagick is installed")
except:
    print("Installing ImageMagick...")
    !apt-get update -qq && apt-get install -y imagemagick -qq
    print("✓ ImageMagick installed")

✓ ImageMagick is installed


## Clone Repository and Compile Code

In [58]:
import os
from pathlib import Path

# Clone the repository if not already cloned
repo_dir = Path('/content/crescent-moon-visibility')
if not repo_dir.exists():
    !git clone https://github.com/yaaqov/crescent-moon-visibility.git /content/crescent-moon-visibility
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists")

# Change to repository directory
os.chdir(repo_dir)
print(f"✓ Working directory: {os.getcwd()}")

fatal: could not create leading directories of '/content/crescent-moon-visibility': Read-only file system
✓ Repository cloned


FileNotFoundError: [Errno 2] No such file or directory: '/content/crescent-moon-visibility'

In [59]:
# Compile the visibility calculator to match map_nasa.png resolution (3600x2160)
# This will be resized to 3840x2160 to match the base map
!gcc -Xpreprocessor -fopenmp -I/opt/homebrew/opt/libomp/include -L/opt/homebrew/opt/libomp/lib -lomp -O3 -Wall -Werror -o visibility.out -fno-exceptions -DPIXEL_PER_DEGREE_LON=10 -DPIXEL_PER_DEGREE_LAT=12 visibility.cc thirdparty/astronomy.c -lm

if os.path.exists('visibility.out'):
    print("✓ Compilation successful (3600x2160 resolution, will be resized to 3840x2160)")
else:
    print("✗ Compilation failed")

✓ Compilation successful (3600x2160 resolution, will be resized to 3840x2160)


## Calculate New Moon Dates

In [60]:
import datetime
import ephem
from typing import List

def get_new_moons_in_year(year: int) -> List[datetime.date]:
    """Returns a list of new moon dates in a year"""
    new_moons = []
    
    date = ephem.Date(datetime.date(year, 1, 1))
    while date.datetime().year == year:
        date = ephem.next_new_moon(date)
        if date.datetime().year == year:
            new_moons.append(date.datetime().date())
    
    return new_moons

# Set the year you want to calculate
YEAR = 2026

new_moon_dates = get_new_moons_in_year(YEAR)
print(f"New Moon dates in {YEAR}:")
for i, date in enumerate(new_moon_dates, 1):
    print(f"{i:2d}. {date.strftime('%Y-%m-%d (%B %d)')}")

New Moon dates in 2026:
 1. 2026-01-18 (January 18)
 2. 2026-02-17 (February 17)
 3. 2026-03-19 (March 19)
 4. 2026-04-17 (April 17)
 5. 2026-05-16 (May 16)
 6. 2026-06-15 (June 15)
 7. 2026-07-14 (July 14)
 8. 2026-08-12 (August 12)
 9. 2026-09-11 (September 11)
10. 2026-10-10 (October 10)
11. 2026-11-09 (November 09)
12. 2026-12-09 (December 09)


## Generate Visibility Maps

In [61]:
import subprocess
from IPython.display import Image, display, HTML
import time

def generate_visibility_map(date_str: str, method: str = 'yallop', time_type: str = 'evening') -> str:
    """
    Generate a visibility map for a specific date.
    
    Args:
        date_str: Date in YYYY-MM-DD format
        method: Calculation method (yallop, odeh, etc.)
        time_type: evening or morning
    
    Returns:
        Path to the generated PNG file
    """
    output_file = f"{date_str}.png"
    
    # Run the visibility calculator
    result = subprocess.run(
        ['./visibility.out', date_str, 'map', time_type, method, output_file],
        capture_output=True,
        text=True
    )
    
    if result.returncode != 0:
        print(f"Error generating map: {result.stderr}")
        return None
    
    # Resize to match base map resolution (3840x2160) before compositing
    subprocess.run(
        ['magick', output_file, '-resize', '3840x2160!', output_file],
        check=True,
        capture_output=True
    )
    
    # Composite with base map (using map_nasa.png which matches resolution)
    subprocess.run(
        ['composite', '-blend', '60', output_file, 'map_nasa.png', output_file],
        check=True,
        capture_output=True
    )
    
    # Create legend in bottom right corner with date and visibility zones
    # Using bullet dots (●) for perfect alignment
    # Order: WORST to BEST visibility
    legend_commands = [
        'magick', output_file,
        
        # Semi-transparent white background box (bottom-right corner)
        '-fill', 'rgba(255,255,255,0.90)',
        '-draw', 'rectangle 3100,1820 3820,2140',
        
        # Date at the top of legend (bold)
        '-fill', 'black',
        '-pointsize', '36',
        '-font', 'Helvetica-Bold',
        '-gravity', 'NorthWest',
        '-annotate', '+3120+1840', date_str,
        
        # Legend title
        '-pointsize', '26',
        '-annotate', '+3120+1885', 'Visibility Zones:',
        
        # Set font for legend items
        '-pointsize', '22',
        '-font', 'Helvetica',
        
        # Dark blue/No color - not visible even with telescope (WORST)
        '-fill', '#0080A0',
        '-annotate', '+3120+1938', '●',
        '-fill', 'black',
        '-annotate', '+3160+1938', 'Not visible (even with telescope)',
        
        # Yellow - visible only with optical aid
        '-fill', '#FFCC00',
        '-annotate', '+3120+1978', '●',
        '-fill', 'black',
        '-annotate', '+3160+1978', 'Visible only with optical aid',
        
        # Light cyan - visible to experienced observer with difficulty
        '-fill', '#00E6E6',
        '-annotate', '+3120+2018', '●',
        '-fill', 'black',
        '-annotate', '+3160+2018', 'Visible to experienced observer',
        
        # Green - easily visible to naked eye (BEST)
        '-fill', '#B3B300',
        '-annotate', '+3120+2058', '●',
        '-fill', 'black',
        '-annotate', '+3160+2058', 'Easily visible to naked eye',
        
        output_file
    ]
    
    subprocess.run(legend_commands, check=True, capture_output=True)
    
    # Optimize for web: reduce file size while maintaining quality
    # Use pngquant for compression (66% size reduction typical)
    subprocess.run(
        ['pngquant', '--quality=85-95', '--speed', '1', '--strip', '--force', '--output', output_file, output_file],
        check=True,
        capture_output=True
    )
    
    # Add interlacing for progressive loading on websites
    subprocess.run(
        ['magick', output_file, '-interlace', 'PNG', output_file],
        check=True,
        capture_output=True
    )
    
    return output_file

print("✓ Functions defined")

✓ Functions defined


## Process All New Moons (3 Days Each)

In [62]:
# Configuration
DAYS_TO_PROCESS = 3  # Process first 3 days after each new moon
METHOD = 'yallop'     # Calculation method
TIME_TYPE = 'evening' # Evening or morning

print(f"Processing {len(new_moon_dates)} new moons for {YEAR}")
print(f"Generating {DAYS_TO_PROCESS} days per new moon = {len(new_moon_dates) * DAYS_TO_PROCESS} total maps\n")

generated_maps = []

for moon_idx, new_moon_date in enumerate(new_moon_dates, 1):
    print(f"\n{'='*60}")
    print(f"Processing New Moon {moon_idx}/{len(new_moon_dates)}: {new_moon_date.strftime('%B %d, %Y')}")
    print(f"{'='*60}")
    
    for day_offset in range(DAYS_TO_PROCESS):
        current_date = new_moon_date + datetime.timedelta(days=day_offset)
        date_str = current_date.strftime('%Y-%m-%d')
        
        print(f"\n  Day {day_offset + 1}: {date_str}")
        
        start_time = time.time()
        output_file = generate_visibility_map(date_str, METHOD, TIME_TYPE)
        elapsed = time.time() - start_time
        
        if output_file and os.path.exists(output_file):
            generated_maps.append((current_date, output_file))
            file_size = os.path.getsize(output_file) / (1024 * 1024)  # MB
            print(f"  ✓ Generated {output_file} ({file_size:.2f} MB) in {elapsed:.1f}s")
        else:
            print(f"  ✗ Failed to generate map for {date_str}")

print(f"\n{'='*60}")
print(f"✓ Complete! Generated {len(generated_maps)} visibility maps")
print(f"{'='*60}")

Processing 12 new moons for 2026
Generating 3 days per new moon = 36 total maps


Processing New Moon 1/12: January 18, 2026

  Day 1: 2026-01-18
  ✓ Generated 2026-01-18.png (3.89 MB) in 46.6s

  Day 2: 2026-01-19
  ✓ Generated 2026-01-19.png (3.07 MB) in 48.7s

  Day 3: 2026-01-20
  ✓ Generated 2026-01-20.png (3.25 MB) in 49.0s

Processing New Moon 2/12: February 17, 2026

  Day 1: 2026-02-17
  ✓ Generated 2026-02-17.png (3.80 MB) in 45.9s

  Day 2: 2026-02-18
  ✓ Generated 2026-02-18.png (2.84 MB) in 50.1s

  Day 3: 2026-02-19
  ✓ Generated 2026-02-19.png (3.37 MB) in 49.3s

Processing New Moon 3/12: March 19, 2026

  Day 1: 2026-03-19
  ✓ Generated 2026-03-19.png (3.02 MB) in 46.1s

  Day 2: 2026-03-20
  ✓ Generated 2026-03-20.png (3.21 MB) in 46.7s

  Day 3: 2026-03-21
  ✓ Generated 2026-03-21.png (3.42 MB) in 46.8s

Processing New Moon 4/12: April 17, 2026

  Day 1: 2026-04-17
  ✓ Generated 2026-04-17.png (3.54 MB) in 44.4s

  Day 2: 2026-04-18
  ✓ Generated 2026-04-18.png (3.02 

## Display Results

In [ ]:
# Display all generated maps
print(f"Displaying {len(generated_maps)} visibility maps:\n")

for date, image_path in generated_maps:
    display(HTML(f"<h3>{date.strftime('%B %d, %Y (%A)')}</h3>"))
    display(Image(filename=image_path, width=1200))  # Display at reasonable size for viewing
    print(f"Full resolution: {image_path}\n")

## Download All Maps as ZIP

In [ ]:
import zipfile
from google.colab import files

# Create a zip file with all generated maps
zip_filename = f"crescent_visibility_{YEAR}.zip"

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for date, image_path in generated_maps:
        zipf.write(image_path, arcname=os.path.basename(image_path))

zip_size = os.path.getsize(zip_filename) / (1024 * 1024)  # MB
print(f"✓ Created {zip_filename} ({zip_size:.2f} MB) with {len(generated_maps)} images")

# Download the zip file
files.download(zip_filename)
print(f"✓ Download started for {zip_filename}")

## Advanced: Custom Date Range

In [ ]:
# Generate maps for specific dates
custom_dates = [
    '2026-01-18',
    '2026-01-19',
    '2026-01-20'
]

print("Generating custom date maps:\n")
for date_str in custom_dates:
    print(f"Processing {date_str}...")
    output_file = generate_visibility_map(date_str)
    if output_file:
        print(f"✓ Generated {output_file}")
        display(Image(filename=output_file, width=1200))
    else:
        print(f"✗ Failed")
    print()

## Summary Statistics

In [ ]:
import pandas as pd

# Create a summary table
summary_data = []
for date, image_path in generated_maps:
    file_size = os.path.getsize(image_path) / (1024 * 1024)  # MB
    summary_data.append({
        'Date': date.strftime('%Y-%m-%d'),
        'Day of Week': date.strftime('%A'),
        'Month': date.strftime('%B'),
        'File': os.path.basename(image_path),
        'Size (MB)': f"{file_size:.2f}"
    })

df = pd.DataFrame(summary_data)
display(HTML(f"<h3>Generated {len(generated_maps)} Visibility Maps for {YEAR}</h3>"))
display(df)

total_size = sum(os.path.getsize(img) for _, img in generated_maps) / (1024 * 1024)
print(f"\nTotal size: {total_size:.2f} MB")
print(f"Average size per map: {total_size/len(generated_maps):.2f} MB")